In [98]:
import pandas as pd
import requests
import io
import unidecode


In [64]:
filenames = ["CasosColombia_01May.csv",
"CasosColombia_03May.csv",
"CasosColombia_04May.csv",
"CasosColombia_05May.csv",
"CasosColombia_10Abril.csv",
"CasosColombia_11Abril.csv",
"CasosColombia_12Abril.csv",
# "CasosColombia_13Abril.csv",
"CasosColombia_14Abril.csv",
"CasosColombia_15Abril.csv",
"CasosColombia_16Abril.csv",
"CasosColombia_17Abril.csv",
# "CasosColombia_17Abril12pm.csv",
"CasosColombia_18Abril.csv",
"CasosColombia_19Abril.csv",
"CasosColombia_1Abril.csv",
"CasosColombia_20Abril.csv",
"CasosColombia_21Abril.csv",
"CasosColombia_22Abril.csv",
"CasosColombia_23Abril.csv",
"CasosColombia_23marzo.csv",
"CasosColombia_24Abril.csv",
"CasosColombia_24marzo.csv",
"CasosColombia_25Abril.csv",
"CasosColombia_25Marzo.csv",
"CasosColombia_26Abril.csv",
"CasosColombia_26marzo.csv",
"CasosColombia_27Abril.csv",
"CasosColombia_27marzo.csv",
"CasosColombia_28Abril.csv",
"CasosColombia_28marzo.csv",
"CasosColombia_29Abril.csv",
"CasosColombia_29marzo.csv",
"CasosColombia_2Abril.csv",
"CasosColombia_30Abril.csv",
"CasosColombia_30marzo.csv",
"CasosColombia_31marzo.csv",
"CasosColombia_3Abril.csv",
"CasosColombia_4Abril.csv",
"CasosColombia_5Abril.csv",
"CasosColombia_6Abril.csv",
"CasosColombia_7Abril.csv",
"CasosColombia_8Abril.csv",
"CasosColombia_9Abril.csv"]

In [110]:
uci_all = pd.DataFrame()
for filename in filenames:
#     print(filename)
    url = f"https://raw.githubusercontent.com/guberney/Covid_Colombia/master/Covid_datos/ReporteDiarios/{filename}"
    s=requests.get(url).content
    df=pd.read_csv(io.StringIO(s.decode('utf-8')))
    df = df.rename(columns={"atención":"atencion",
                            "Atención**": "atencion",
                            "Atención": "atencion",
                            "Edad": "edad",
                            "Ciudad de ubicación": "ciudad"})
    df["atencion"] = df["atencion"].str.upper()
    df["ciudad"] = df["ciudad"].apply(lambda x: unidecode.unidecode(x.upper()))
    df = df.replace("BOGOTA D.C.", "BOGOTA")
    uci = df[df["atencion"] == "HOSPITAL UCI"]
    nombre_fecha = filename
    nombre_fecha = nombre_fecha.replace("CasosColombia_", "").replace(".csv", "")
    nombre_fecha = nombre_fecha.replace("May", "-05").replace("Abril", "-04").replace("marzo", "-03").replace("Marzo", "-03")
    nombre_fecha = nombre_fecha + "-2020"
    uci_by_city = uci.groupby("ciudad")["edad"].count().to_frame(nombre_fecha).T

    uci_all = uci_all.append(uci_by_city)

uci_all = uci_all.reset_index()
uci_all = uci_all.rename(columns={"index":"fecha"})
uci_all["fecha"] = pd.to_datetime(uci_all["fecha"], dayfirst=True)
uci_all = uci_all.sort_values("fecha")
uci_all["fecha"] = uci_all["fecha"].astype(str)

In [117]:
uci_all.columns

Index(['fecha', 'AGUACHICA', 'BARRANQUILLA', 'BELLO', 'BOGOTA', 'BUENAVENTURA',
       'CALI', 'CARTAGENA DE INDIAS', 'CUCUTA', 'DOSQUEBRADAS', 'FLORIDA',
       'IBAGUE', 'IPIALES', 'ITAGUI', 'MALAMBO', 'MEDELLIN', 'MOSQUERA',
       'NEIVA', 'PALERMO', 'PALMIRA', 'PASTO', 'PEREIRA', 'SAN CRISTOBAL',
       'SAN JUAN DEL CESAR', 'SANTA MARTA', 'VALLEDUPAR', 'VILLAVICENCIO',
       'YUMBO', 'DAGUA', 'CIENAGA', 'COPACABANA', 'SOLEDAD', 'CARTAGENA',
       'ENVIGADO', 'ESPINAL', 'FLORIDABLANCA', 'SAN FRANCISCO', 'TENJO',
       'ZIPAQUIRA', 'VILLAMARIA', 'VILLA DEL ROSARIO', 'ARMENIA', 'JAMUNDI',
       'PUERTO COLOMBIA', 'SAN MARTIN', 'RIOHACHA', 'SONSON', 'TUNJA',
       'TURBACO', 'VILLAPINZON', 'CALAMAR', 'EL COLEGIO', 'SIBATE',
       'RIO DE ORO', 'PUEBLOVIEJO', 'MELGAR', 'POPAYAN', 'RIONEGRO', 'TOGUI',
       'BARRANCABERMEJA'],
      dtype='object')

In [120]:
uci_all.to_csv("ciudad-uci.csv")